In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:

#dataset of docs taken from jupyter-dev using:
#     from gaspy import gasdb, defaults
#     import warnings
#     warnings.filterwarnings('ignore')

#     filters = defaults.adsorption_filters('CO')
#     #filters['results.energy'] = {'$gt': -3.5, '$lt': 9.0}
#     #filters['processed_data.movement_data.max_adsorbate_movement']['$lt'] = 4.0 #specifically for OOH vs OH

#     # Establish the fingerprints that are needed for the preprocessing
#     fingerprints = {}
#     fingerprints['atoms']='$atoms'
#     fingerprints['results']='$results'
#     fingerprints['max_surface_movement']='$processed_data.movement_data.max_surface_movement'
#     fingerprints['adsorption_site'] = '$initial_configuration.atoms.atoms'
#     # Pull the documents and then modify them so that they'll work with the preprocessor
#     docs = gasdb.get_adsorption_docs(['CO'],extra_fingerprints=fingerprints, filters=filters)

#CO_docs = pickle.load(open('/home/zulissi/CO_docs_200.pkl','rb'))
    

# with open('/home/zulissi/CO_docs_200.pkl','wb') as fhandle:
#     pickle.dump(CO_docs,fhandle)

Transform block to turn a GASpy doc into the necessary structure for CGCNN

In [5]:
from torch.utils.data import Dataset, DataLoader
import mongo
from pymatgen.io.ase import AseAtomsAdaptor
import pickle
from cgcnn.data import StructureData
import torch
import numpy as np
from sklearn.base import TransformerMixin
import tqdm
import random

class ListDataset(Dataset):
    def __init__(self, list_in):
        self.list = list_in
        
    def __len__(self):
        return len(self.list)

    def __getitem__(self, idx):
        return self.list[idx]

class StructureDataTransformer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
        return
    
    def transform(self,X):
        target_list = scaler.transform(np.array([doc['energy'] for doc in X]).reshape(-1, 1))
        structure_list = [mongo.make_atoms_from_doc(doc) for doc in X]
        SD = StructureData(structure_list, target_list, *self.args, **self.kwargs)
        return SD

    def fit(self,*_):
        return self
    
CO_docs = pickle.load(open('/home/zulissi/CO_docs.pkl','rb'))
random.shuffle(CO_docs)
CO_docs = CO_docs[:2000]

energies = np.array([doc['energy'] for doc in CO_docs])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(energies.reshape(-1, 1))
# CO_docs = pickle.load(open('/home/zulissi/CO_docs_200.pkl','rb'))


SDT = StructureDataTransformer(atom_init_loc='/home/zulissi/software/cgcnn_sklearn/atom_init.json',
                              max_num_nbr=9,
                              radius=1,
                              use_tag=True,
                              use_fixed_info=True)

SDT = SDT.transform(CO_docs)

import multiprocess as mp
with mp.Pool(4) as pool:
    SDT_list = ListDataset(list(tqdm.tqdm(pool.imap(lambda x: SDT[x],range(len(SDT)),chunksize=40),total=len(SDT))))

#SDT_list = ListDataset(list(tqdm.tqdm(map(lambda x: SDT[x],range(len(SDT))),total=len(SDT))))

# SDT_list= map([a for a in tqdm.tqdm(SDT_X)])

100%|██████████| 2000/2000 [03:34<00:00,  9.34it/s] 


In [6]:
from cgcnn.model import CrystalGraphConvNet
from torch import nn
from skorch import NeuralNetRegressor
from torch.utils.data import Dataset, DataLoader
from cgcnn.data import collate_pool

cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

#This is a little weird, one of the parameters basically needs to match the results 
# that are coming from the transform function, should be fixable 
structures, _ = SDT_list[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]


CGCNN model with skorch to make it sklearn compliant

In [7]:
from torch.optim import Adam
from sklearn.model_selection import ShuffleSplit
#from skorch.callbacks import Checkpoint, TrainEndCheckpoint

#cp = Checkpoint(dirname='exp1')
net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    module__atom_fea_len=46,
    module__h_fea_len=83,
    module__n_conv=8,
    module__n_h=4,
    iterator_train__batch_size=214,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=4,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=4,
    iterator_valid__collate_fn = collate_pool,
    max_epochs=292,
    lr=np.exp(-5.18),
    optimizer=Adam,    
    device=device,
    criterion=torch.nn.L1Loss,
#    callbacks=[cp]
)

In [8]:
from sklearn.pipeline import make_pipeline
# pipe = make_pipeline(SDT, net)


#y, energies
target_list = [doc['energy'] for doc in CO_docs]
y = torch.Tensor(np.array(target_list).reshape((len(target_list),1)))

#This is hacky, I'm not sure if y is actually necessary, I think it's just buried
# in the CO_docs, but need to check

net.fit(SDT_list,y=y)

OSError: [Errno 12] Cannot allocate memory